In [31]:
import os
import json
import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm 

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from indicnlp.tokenize import indic_tokenize
from nltk.corpus import stopwords

from deep_translator import GoogleTranslator

In [32]:
def split_long_sentence(sentence, max_length=3000):
    parts = []
    for i in range(0, len(sentence), max_length):
        part = sentence[i:i + max_length]
        parts.append(part)

    return parts

In [33]:
def convertToEnglish(sentences):
    eng = []
    for sentence_array in sentences:
        sentence = ''
        engText=''
        for i in range(len(sentence_array)):
            sentence = sentence + sentence_array[i]
        if len(sentence)>2000:
            small_sentences = split_long_sentence(sentence, max_length=2000)
            engPart = ''
            for i in range(len(small_sentences)):
                current_translate = GoogleTranslator(source='hi', target='en').translate(small_sentences[i])
                engPart = engPart + current_translate
        else:
            engPart = GoogleTranslator(source='hi', target='en').translate(sentence)
        engText=engText +engPart
        eng.append(engText)
    return eng

In [34]:
stop=open('stopwords/hindi.txt')
hindi_stop_words=[]
for x in stop:
    hindi_stop_words.append(x.strip("\n"))

def tokenize_hindi(sentence):
    tokens = indic_tokenize.trivial_tokenize(sentence, lang='hi')
    
    filtered_tokens = [token for token in tokens if token not in hindi_stop_words]

    return filtered_tokens

english_stop_words = set(stopwords.words('english'))
def tokenize_english(sentence):
    tokens = word_tokenize(sentence)
    
    filtered_tokens = [token for token in tokens if token not in english_stop_words]
    
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in lemmatized_tokens]

    stemmed_tokens = [token for token in stemmed_tokens if token.isalpha()]
    
    return stemmed_tokens

In [35]:
def process(file,sampleCount):
    print(f"Current file:{file}")
    with open(file) as f:
        data = json.load(f)
    df = pd.DataFrame(data)

    df=df.transpose()
    df = df.head(sampleCount)

    facts_and_arguments = []
    for i, row in df.iterrows():
        facts_and_arguments.append(df.loc[i]['segments']['facts-and-arguments'])
    df['facts-and-arguments'] = facts_and_arguments
    
    

    engSentencesCollection = []
    sentencesCollection =[]
    tokensCollection = []
    engTokensCollection = []
    judgeOpinionCollection = []
    for i, row in tqdm(df.iterrows(),total=len(df)):
        judge_sentences=[]
        sentences=[]

        paras = df.loc[i]['judge-opinion']
        for para in paras:
            sent = para.split('।')
            sent = [i for i in sent if len(i)!=0 and i!=' ']
            judge_sentences.extend(sent)
        
        paras = df.loc[i]['facts-and-arguments']
        for para in paras:
            sent = para.split('।')
            sent = [i for i in sent if len(i)!=0 and i!=' ']
            sentences.extend(sent)
        sentences = [i.strip() for i in sentences]

        english_sentences = convertToEnglish(sentences)
        tokens = [tokenize_hindi(sentence ) for sentence in sentences]
        english_tokens = [tokenize_english(sentence ) for sentence in english_sentences]
        
        judgeOpinionCollection.append(judge_sentences)
        tokensCollection.append(tokens)
        sentencesCollection.append(sentences)
        engTokensCollection.append(english_tokens)
        engSentencesCollection.append(english_sentences)

    df.drop(columns=['segments','district','case_number'],inplace=True)
    df['sentences'] = sentencesCollection
    df['tokens'] = tokensCollection
    df['english_tokens'] = engTokensCollection
    df['sentences_english'] = engSentencesCollection
    df['judge-opinion'] = judgeOpinionCollection
    return df

In [36]:
base_path = "./data/raw/"
write_path = "./data/summary/"
files = os.listdir(base_path)
try:
    os.mkdir(write_path)
except:
    pass
files = [f for f in files if ".json" in f]

dfs=[]

sampleCount = [1,1,1]
for file,count in zip(files,sampleCount):
    dfs.append(process(base_path+file, count))

df = pd.concat(dfs, ignore_index=True)
df.to_csv(f"{write_path}/data.csv",index=False)

Current file:./data/raw/test.json


  0%|          | 0/1 [00:00<?, ?it/s]

['जमानत प्रार्थनापत्र के समर्थन में अभियुक्त के पैरोकार शरीफ उर्फशफीक ने शपथ पत्र प्रस्तुत किया है', 'अभियोजन कथानक संक्षेप में इस प्रकार है कि वादी मुकदमा लक्ष्मीनरायन ने थाना शाहगंज <नाम> दिनाक 09.08.2020 को इस आशय की तहरीर दी किउसका स्कूटर मेस्ट्रो रंग स्काई ब्लू नम्बर यूपी.80 डी.डी.2730 दिनांक 22.07.2020 कोरात्रि 9.30 बजे उसके निवास <नाम> ताला लगा खड़ी थी, तभी समय करीब 10 बजेउसने <नाम> तो उसके उक्त स्कूटर को अज्ञात चोर चोरी करके ले गया है', 'उसकेपड़ौस में लगे सी.सी.टी.वी.कैमरे में देखने <नाम> तीन अज्ञात चोर 9.54 पी.एम. <नाम> स्कूटरमें चाबी लगाकर चोरी करके स्कूटर ले जाते दिखाई <नाम> रहे थे', 'फुटेज की क्लिपउसके पास मौजूद है', 'इस घटना की जानकारी उसने तत्काल 112 नं0 <नाम> <नाम> दीशी', 'काफी खोजबीन करने <नाम> उसकी उक्त स्कूटी नहीं <नाम> है', 'वादी मुकदमा की उक्त तहरीर के आधार <नाम> थाना शाहगंज <नाम> अज्ञातके विरूद्ध मु0अ0सं0 3612020 अंतर्गत <नाम> 379 भा0द0सं0 पंजीकृत हुआ', '<नाम> मेंप्रकरण में अभियुक्त के पकड़े जाने <नाम> <नाम> 411 भा0द0सं0 की बढोत्तरी की गयी है', 'अभियुक्त के विद्वान

  0%|          | 0/1 [00:00<?, ?it/s]

['अग्रिम जमानत प्रार्थनापत्र के समर्थन में प्रार्थी, अभियुक्त <नाम> <नाम> शर्माका शपथपत्र प्रस्तुत किया गया है कि जिसमें यह वर्णित है कि अभियुक्तगण का यह प्रथमअग्रिम जमानत प्रार्थना पत्र है, अन्य कोई अग्रिम जमानत प्रार्थनापत्र किसी अन्य न्यायालय यामाननीय उच्च न्यायालय में विचाराधीन नहीं है', 'अभियोजन कथानक सक्षेंप में इस प्रकार है कि वादी <नाम> <नाम> के द्वाराथाना खेरागढ <नाम> <नाम> तहरीर इस आशय की दी गईं कि-दिनांक 20.07.214 को समयकरीब 09.00 बजे वादी के मकान के सामने विपक्षी द्वारा करीब 50 मीटर की दूरी परसमरसेविल का वोरिंग <नाम> रहा था', 'वादी ने <नाम> किया कि यहयू <नाम> उसके हैंडपम्प कावोरिग हैं इसी बात से क्षुब्थ होकर विपक्षी राजवीर, गोकुल, <नाम> एक साथ वादी के घर परआ धमके, वादी के ऊपर गोकुल द्वारा लोहे की सरिया से प्रहार किया, लेकिन बीच में वादीके पिता <नाम> <नाम> उक्त सरिया को पकडा, सरिया छूट जाने <नाम> पुनः सरिया से प्रहारवादी के पिता रोशल <नाम> <नाम> ही <नाम> दिया', 'जिससे सिर फट गया', 'सिर में काफी घाव होगया', 'शोरगुल सुनकर मौहल्ले वाले आ गये', 'इतने में उक्त विपक्षी फरार हो गये

  0%|          | 0/1 [00:00<?, ?it/s]

['जमानत प्रार्थना पत्र प्रार्थी/अभियुक्त की मौसी व पैरोकारश्रीसती भगवान <नाम> चाहर के शपथ पत्र से समर्थित है', 'संक्षेप में अभियोजन कथानक के अनुसार वादी मुकदमाविश्वनाथ द्वारा थाने <नाम> प्रथम सूचना रिपोर्ट इस आशय की दर्ज करायी गयीकि दिनाक 01.10.2019 को <नाम> को किरावली से ट्रक संख्या-आरo०जे011/ जी0बी० 2738 से सैंया जा रहा था कि ग्राम मोरी के पास समय 11:30 बजे <नाम> को मोटरसाईकिल सवार तीन लड़कों ने उसके ट्रक से उसकामोबाइल ओटो ए३7 छीन लिया और मोटरसाईकिल संख्या-यूएपी0 80/ई0सी0 5872 से भाग गए हैं', 'प्रार्थी / अभियुक्त की ओर से विद्वान अधिवक्ता द्वारा यह कहागया है कि प्रार्थी को मामले में झूठा फंसाया गया है, उसने कोई अपराध नहींकिया है, वह निर्दोष है', 'प्रार्थी का नाम दौरान विवचेना <नाम> में आया है', '्रार्शी दिनाक 25.11.2019 से जिला कारागार में निरूद्ध है', 'प्रार्थी प्रथम सूचनारिपोर्ट में नामजद नहीं है', 'प्रार्थी से कोई बरामदगी नहीं है, झूठी बरामदगी (2)न्यायालय विशेष न्यायाधीश (दएप्र0कषेत्र)/अपर सत्र न्यायाधीश, न्यायालय संख्या-03, आगरा', 'जमानत प्रार्थना पत्र संख्या : 7218/2019भूरा बनाम